In [121]:
# import required libraries and settings
import pandas as pd
import glob
import numpy as np
import csv
import openai
import random
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import re
import time 
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import dotenv
import ipywidgets as widgets
from IPython.display import display
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from sklearn.gaussian_process import GaussianProcessRegressor
load_dotenv()
api_key = os.getenv('API_KEY')
openai.api_key = api_key
%run ID_settings.ipynb
display(settings)

In [124]:
%run ID_Utilities.ipynb
display(Prompts)    

SUMMARY
The design target is to achieve a strength of  65.26 MPa within  10  development cycles.
The Experiment is repeated  10  times using the  gpt-3.5-turbo  model and the prompt strategy:  None .
There are  1  formulations above or equal to desired_strength.


## Run Experiment

In [ ]:
# Iterate over the different temperature settings
for temp in temperatures:

    
    # Repeat the whole experiment n times
    for experiment in range(NrOfExper):
        print(f"\n---\nStarting experiment {experiment+1}...\n---")
        training_data = []
        current_strength = 0.0
        iterations = 0

        # System message including both the role prompt and context
        system_message = system_role_prompt.value  + '\n' + context_prompt.value

        while iterations < budget:
            iterations += 1
            print(f"\n---\nStarting iteration {iterations} at temp {temp}...")

            # Start with the system message
            messages = [
                {"role": "system", "content": system_message}
            ]

            # Add the training data to the messages
            if training_data:
                messages.append({"role": "assistant", "content": "Previously, we have tested these formulations with the following result:\n" + "\n".join(training_data)})
            if training_data:
                # Add the iteration prompt
                messages.append({"role": "user", "content": iterate_prompt.value})
            
            if conversation_dropdown.value == 'Complete':
                print("--- Conversation History ---")
                for msg in messages:
                    print(f"{msg['role']}: {msg['content']}")

            response = call_openai_api(messages,temp)
            lab_result = None  # initialize lab_result

            # Check the role and extract the content if role is 'assistant'
            valid_solution = False
            while not valid_solution:
                response = call_openai_api(messages,temp)
                if response['choices'][0]['message']['role'] == "assistant":
                    print('Model Response: ',response['choices'][0]['message']['content'])
                    suggested_solution = parse_solution(response['choices'][0]['message']['content'])
                    if suggested_solution is not None:
                        #print(f"Parse result: {suggested_solution}")
                        lab_result = find_matching_result(formulation_df, suggested_solution)
                        if lab_result:
                            current_strength = lab_result
                            # Add the solution and its lab result to the training data
                            training_data.append(f"{response['choices'][0]['message']['content']} resulted in a strength of {current_strength} MPa.")
                            valid_solution = True
                        else:
                            print(f"Iteration {iterations}: No matching lab result found for suggestion {suggested_solution}")
                            # Add reminder to the user prompt
                            messages[-1]["content"] = iterate_prompt.value + "\nPlease remember to stick to the exact parameter grid! No extrapolation or interpolation of the parameters is allowed!  Use this exact format: 'The formulation is Powderkg = {your estimate}, wc = {your estimate}, materials = {your estimate}, curing = {your estimate}'."
                    else:
                        print(f"Iteration {iterations}: Assistant's response did not contain a valid solution. Trying again.")
                        # Add reminder to the user prompt
                        messages[-1]["content"] = iterate_prompt.value + "\nPlease remember to stick to the exact parameter grid and format: 'The formulation is Powderkg = {your estimate}, wc = {your estimate}, materials = {your estimate}, curing = {your estimate}'."
                else:
                    print(f"Iteration {iterations}: Response not from 'assistant'. Trying again.")
                print('The suggested formulation achieved a strength of ',current_strength,' MPa.')
                if current_strength >= desired_strength:
                    print(f"\nDesired compressive strength of {desired_strength} MPa achieved after {iterations} iterations. The solution is {suggested_solution}.")
                    break
            if current_strength >= desired_strength:
                break

        timestamp = str(int(time.time()))

        # create the file name
        filename = f"Results/ID/{model_dropdown.value}_{prompt_dropdown.value}_prompt_experiment_{experiment+1}_temp_{temp}_target_{int(targ_quant.value)}_%_Dev_Budget_{budget}_{timestamp}.csv"

        # open the file in write mode
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)

            # write the headers
            writer.writerow(["Formulation", "Compressive Strength"])

            # iterate over the training data
            for data in training_data:
                # parse the data to extract formulation and compressive strength
                formulation, strength_str = data.split(" resulted in a strength of ")
                strength = float(strength_str.split(" ")[0])  # convert string to float
                writer.writerow([formulation, strength])

        print(f"Data for experiment {experiment+1} and temp {temp} successfully saved to {filename}.")



---
Starting experiment 1...
---

---
Starting iteration 1 at temp 0.0...
--- Conversation History ---
system: ////You are a powerful concrete formulation prediction model tasked with finding the best concrete formulation that maximizes compressive strength. Your predictions will be validated in the Laboratory and you will receive the real-world performance. You will learn from the feedback provided to improve your previous suggestions to find a perfect mix design. Make sure that every formulation lies on this parameter grid: //Materials: Fly-Ash/GGBFS at a ratio: 0.7/0.3, 0.6/0.4, 0.5/0.5 //water-to-cement (WC) ratio: 0.45, 0.5, 0.55, 0.6 //curing: Ambient curing/Heat curing //powder content in kg: 360, 370, 380, 390,400, 410, 420, 430, 440, 450 //You are able to incorporate design rules and lab validations to improve your predictions. You can only answer in this exact format with no additional explanations or context: 'The formulation is Powderkg = {your estimate}, wc = {your estima

Model Response:  The formulation is Powderkg = 410, wc = 0.5, materials = 0.7/0.3, curing = Heat curing.
The suggested formulation achieved a strength of  43.35  MPa.

---
Starting iteration 7 at temp 0.0...
--- Conversation History ---
system: ////You are a powerful concrete formulation prediction model tasked with finding the best concrete formulation that maximizes compressive strength. Your predictions will be validated in the Laboratory and you will receive the real-world performance. You will learn from the feedback provided to improve your previous suggestions to find a perfect mix design. Make sure that every formulation lies on this parameter grid: //Materials: Fly-Ash/GGBFS at a ratio: 0.7/0.3, 0.6/0.4, 0.5/0.5 //water-to-cement (WC) ratio: 0.45, 0.5, 0.55, 0.6 //curing: Ambient curing/Heat curing //powder content in kg: 360, 370, 380, 390,400, 410, 420, 430, 440, 450 //You are able to incorporate design rules and lab validations to improve your predictions. You can only answ

Model Response:  The formulation is Powderkg = 370, wc = 0.5, materials = 0.6/0.4, curing = Ambient curing.
The suggested formulation achieved a strength of  48.47  MPa.
Data for experiment 1 and temp 0.0 successfully saved to Results/ID/gpt-3.5-turbo_None_prompt_experiment_1_temp_0.0_target_100_%_Dev_Budget_10_1690370969.csv.

---
Starting experiment 2...
---

---
Starting iteration 1 at temp 0.0...
--- Conversation History ---
system: ////You are a powerful concrete formulation prediction model tasked with finding the best concrete formulation that maximizes compressive strength. Your predictions will be validated in the Laboratory and you will receive the real-world performance. You will learn from the feedback provided to improve your previous suggestions to find a perfect mix design. Make sure that every formulation lies on this parameter grid: //Materials: Fly-Ash/GGBFS at a ratio: 0.7/0.3, 0.6/0.4, 0.5/0.5 //water-to-cement (WC) ratio: 0.45, 0.5, 0.55, 0.6 //curing: Ambient curi

Model Response:  The formulation is Powderkg = 450, wc = 0.45, materials = 0.6/0.4, curing = Ambient curing.
The suggested formulation achieved a strength of  45.88  MPa.

---
Starting iteration 6 at temp 0.0...
--- Conversation History ---
system: ////You are a powerful concrete formulation prediction model tasked with finding the best concrete formulation that maximizes compressive strength. Your predictions will be validated in the Laboratory and you will receive the real-world performance. You will learn from the feedback provided to improve your previous suggestions to find a perfect mix design. Make sure that every formulation lies on this parameter grid: //Materials: Fly-Ash/GGBFS at a ratio: 0.7/0.3, 0.6/0.4, 0.5/0.5 //water-to-cement (WC) ratio: 0.45, 0.5, 0.55, 0.6 //curing: Ambient curing/Heat curing //powder content in kg: 360, 370, 380, 390,400, 410, 420, 430, 440, 450 //You are able to incorporate design rules and lab validations to improve your predictions. You can only 

Model Response:  The formulation is Powderkg = 390, wc = 0.45, materials = 0.5/0.5, curing = Ambient curing.
The suggested formulation achieved a strength of  56.1  MPa.

---
Starting iteration 10 at temp 0.0...
--- Conversation History ---
system: ////You are a powerful concrete formulation prediction model tasked with finding the best concrete formulation that maximizes compressive strength. Your predictions will be validated in the Laboratory and you will receive the real-world performance. You will learn from the feedback provided to improve your previous suggestions to find a perfect mix design. Make sure that every formulation lies on this parameter grid: //Materials: Fly-Ash/GGBFS at a ratio: 0.7/0.3, 0.6/0.4, 0.5/0.5 //water-to-cement (WC) ratio: 0.45, 0.5, 0.55, 0.6 //curing: Ambient curing/Heat curing //powder content in kg: 360, 370, 380, 390,400, 410, 420, 430, 440, 450 //You are able to incorporate design rules and lab validations to improve your predictions. You can only 

...

Model Response:  The formulation is Powderkg = 440, wc = 0.45, materials = 0.5/0.5, curing = Heat curing.
The suggested formulation achieved a strength of  55.78  MPa.

---
Starting iteration 9 at temp 0.35...
--- Conversation History ---
system: ////You are a powerful concrete formulation prediction model tasked with finding the best concrete formulation that maximizes compressive strength. Your predictions will be validated in the Laboratory and you will receive the real-world performance. You will learn from the feedback provided to improve your previous suggestions to find a perfect mix design. Make sure that every formulation lies on this parameter grid: //Materials: Fly-Ash/GGBFS at a ratio: 0.7/0.3, 0.6/0.4, 0.5/0.5 //water-to-cement (WC) ratio: 0.45, 0.5, 0.55, 0.6 //curing: Ambient curing/Heat curing //powder content in kg: 360, 370, 380, 390,400, 410, 420, 430, 440, 450 //You are able to incorporate design rules and lab validations to improve your predictions. You can only an

Model Response:  The formulation is Powderkg = 410, wc = 0.55, materials = 0.5/0.5, curing = Ambient curing.
The suggested formulation achieved a strength of  47.28  MPa.

---
Starting iteration 4 at temp 0.35...
--- Conversation History ---
system: ////You are a powerful concrete formulation prediction model tasked with finding the best concrete formulation that maximizes compressive strength. Your predictions will be validated in the Laboratory and you will receive the real-world performance. You will learn from the feedback provided to improve your previous suggestions to find a perfect mix design. Make sure that every formulation lies on this parameter grid: //Materials: Fly-Ash/GGBFS at a ratio: 0.7/0.3, 0.6/0.4, 0.5/0.5 //water-to-cement (WC) ratio: 0.45, 0.5, 0.55, 0.6 //curing: Ambient curing/Heat curing //powder content in kg: 360, 370, 380, 390,400, 410, 420, 430, 440, 450 //You are able to incorporate design rules and lab validations to improve your predictions. You can only

Model Response:  The formulation is Powderkg = 440, wc = 0.6, materials = 0.6/0.4, curing = Ambient curing.
The suggested formulation achieved a strength of  46.01  MPa.

---
Starting iteration 9 at temp 0.35...
--- Conversation History ---
system: ////You are a powerful concrete formulation prediction model tasked with finding the best concrete formulation that maximizes compressive strength. Your predictions will be validated in the Laboratory and you will receive the real-world performance. You will learn from the feedback provided to improve your previous suggestions to find a perfect mix design. Make sure that every formulation lies on this parameter grid: //Materials: Fly-Ash/GGBFS at a ratio: 0.7/0.3, 0.6/0.4, 0.5/0.5 //water-to-cement (WC) ratio: 0.45, 0.5, 0.55, 0.6 //curing: Ambient curing/Heat curing //powder content in kg: 360, 370, 380, 390,400, 410, 420, 430, 440, 450 //You are able to incorporate design rules and lab validations to improve your predictions. You can only 

Model Response:  The formulation is Powderkg = 430, wc = 0.5, materials = 0.5/0.5, curing = Ambient curing.
The suggested formulation achieved a strength of  59.75  MPa.

---
Starting iteration 4 at temp 0.35...
--- Conversation History ---
system: ////You are a powerful concrete formulation prediction model tasked with finding the best concrete formulation that maximizes compressive strength. Your predictions will be validated in the Laboratory and you will receive the real-world performance. You will learn from the feedback provided to improve your previous suggestions to find a perfect mix design. Make sure that every formulation lies on this parameter grid: //Materials: Fly-Ash/GGBFS at a ratio: 0.7/0.3, 0.6/0.4, 0.5/0.5 //water-to-cement (WC) ratio: 0.45, 0.5, 0.55, 0.6 //curing: Ambient curing/Heat curing //powder content in kg: 360, 370, 380, 390,400, 410, 420, 430, 440, 450 //You are able to incorporate design rules and lab validations to improve your predictions. You can only 

Model Response:  The formulation is Powderkg = 390, wc = 0.6, materials = 0.6/0.4, curing = Ambient curing.
The suggested formulation achieved a strength of  46.35  MPa.

---
Starting iteration 9 at temp 0.35...
--- Conversation History ---
system: ////You are a powerful concrete formulation prediction model tasked with finding the best concrete formulation that maximizes compressive strength. Your predictions will be validated in the Laboratory and you will receive the real-world performance. You will learn from the feedback provided to improve your previous suggestions to find a perfect mix design. Make sure that every formulation lies on this parameter grid: //Materials: Fly-Ash/GGBFS at a ratio: 0.7/0.3, 0.6/0.4, 0.5/0.5 //water-to-cement (WC) ratio: 0.45, 0.5, 0.55, 0.6 //curing: Ambient curing/Heat curing //powder content in kg: 360, 370, 380, 390,400, 410, 420, 430, 440, 450 //You are able to incorporate design rules and lab validations to improve your predictions. You can only 

### BO Baseline

In [6]:

# Define an acquisition function
def acquisition(X, model, scaler, epsilon=0.01):
    prediction, std_dev = model.predict(scaler.transform(X), return_std=True)
    return -(prediction + epsilon * std_dev)

# load the data
file_path = os.path.join('Data', 'numeric_data.csv')
data = pd.read_csv(file_path)

# Extract features and target
features = ["Powderkg", "Liquidkg", "WC", "Fly_Ash_ratio", "GGBFS_ratio", "temperature"]
target = "fc_28dGroundTruth"

# Initialize kernel and GP model
#kernel = ConstantKernel(1.0, (1e-3, 1e3)) * Matern(length_scale=10, nu=1.5)
kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
gpr = GaussianProcessRegressor(kernel=kernel)

timestamp = str(int(time.time()))

for experiment in range(30):
    print(f"Experiment: {experiment+1}")

    # Randomly sample initial training set
    initial_sample_size = 4
    train_data = data.sample(n=initial_sample_size)
    while train_data[target].max() >= desired_strength:
        train_data = data.sample(n=initial_sample_size)
        
    # All other data is potential test data
    test_data = data.drop(train_data.index)

    # Define scalers outside the loop
    X_scaler = StandardScaler()
    y_scaler = MinMaxScaler()

    for _ in range(10):
        # Fit and transform the data
        X_train = X_scaler.fit_transform(train_data[features])
        y_train = y_scaler.fit_transform(train_data[target].values.reshape(-1, 1))

        # Train GP model
        gpr.fit(X_train, y_train)

        # Use the acquisition function to select the next point from the test set
        X_test = X_scaler.transform(test_data[features])
        acq_values = -acquisition(X_test, gpr, X_scaler)  # negative because we are maximizing

        # Select the point that maximizes the acquisition function
        max_acq_index = np.argmax(acq_values)
        X_next = X_test[max_acq_index]

        # Get the original index from the test_data DataFrame
        original_index = test_data.index[max_acq_index]

        # Append X_next to training data
        train_data = train_data.append(test_data.loc[original_index])

        # Remove X_next from test_data
        test_data = test_data.drop(original_index)

        # If we have found a material with the desired strength, we stop sampling
        if train_data.loc[original_index, target] >= desired_strength:
            break
     
    filename = f"results/BO/experiment_{experiment+1}_initialsample_{initial_sample_size}_experiment_{experiment+1}_target_{int(targ_quant.value)}_%_Dev_Budget_{budget}_{timestamp}.csv"
    train_data.to_csv(filename, index=False)


Experiment: 1


/Users/christophvoelker/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-6-cc8a88686339>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
/Users/christophvoelker/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-6-cc8a88686339>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
/Users/christophvoelker/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature 

Experiment: 2
Experiment: 3


/Users/christophvoelker/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.01. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/christophvoelker/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-6-cc8a88686339>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
/Users/christophvoelker/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.01

Experiment: 4
Experiment: 5


...

/Users/christophvoelker/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.01. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/christophvoelker/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-6-cc8a88686339>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
/Users/christophvoelker/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.01

Experiment: 28
Experiment: 29


<ipython-input-6-cc8a88686339>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
/Users/christophvoelker/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-6-cc8a88686339>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
/Users/christophvoelker/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-6-cc8a88686339>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a futur

Experiment: 30


/Users/christophvoelker/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.01. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/christophvoelker/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-6-cc8a88686339>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
/Users/christophvoelker/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.01

### RF Baseline

In [7]:
from sklearn.preprocessing import StandardScaler
from lolopy.learners import RandomForestRegressor
import numpy as np
import pandas as pd


def acquisition(X, model, scaler):
    '''Acquisition function that finds the maximum of predictions plus uncertainties.'''
    # Get predictions and uncertainties
    preds, stds = model.predict(X, return_std=True)
    # Return predictions plus uncertainties
    return preds + stds

def run_experiment(data, features, target, desired_strength, experiment_num=30, initial_sample_size=4, iteration_num=10):
    # Initialize Random Forest model
    dtr = RandomForestRegressor()

    # Define scaler outside the loop
    X_scaler = StandardScaler()

    for experiment in range(experiment_num):
        print(f"Experiment: {experiment+1}")
        # Randomly sample initial training set
        train_data = data.sample(n=initial_sample_size)

        # All other data is potential test data
        test_data = data.drop(train_data.index)

        for _ in range(iteration_num):
            # Fit and transform the data
            X_train = X_scaler.fit_transform(train_data[features])
            y_train = train_data[target].values
            # Check if number of samples less than 8, if yes, tile them
            if X_train.shape[0] < 8:
                repeat_times = 8 // X_train.shape[0] + 1
                X_train = np.tile(X_train, (repeat_times, 1))
                y_train = np.tile(y_train, repeat_times)
            # Train RF model
            dtr.fit(X_train, y_train)

            # Use the acquisition function to select the next point from the test set
            X_test = X_scaler.transform(test_data[features])
            acq_values = acquisition(X_test, dtr, X_scaler)

            # Select the point that maximizes the acquisition function
            max_acq_index = np.argmax(acq_values)
            X_next = X_test[max_acq_index]

            # Get the original index from the test_data DataFrame
            original_index = test_data.index[max_acq_index]

            # Append X_next to training data
            train_data = train_data.append(test_data.loc[original_index])

            # Remove X_next from test_data
            test_data = test_data.drop(original_index)

            # If we have found a material with the desired strength, we stop sampling
            if train_data.loc[original_index, target] >= desired_strength:
                break
        filename = f"results/RF_ID/experiment_{experiment+1}_RF_initialsample_{initial_sample_size}_experiment_{experiment+1}_target_{int(targ_quant.value)}_%_Dev_Budget_{budget}_{timestamp}.csv"
        train_data.to_csv(filename, index=False)
        
        
# load the data
file_path = os.path.join('Data', 'numeric_data.csv')
data = pd.read_csv(file_path)

# Extract features and target
features = ["Powderkg", "Liquidkg", "WC", "Fly_Ash_ratio", "GGBFS_ratio", "temperature"]
target = "fc_28dGroundTruth"

timestamp = str(int(time.time()))               
run_experiment(data, features, target, desired_strength, experiment_num=30, initial_sample_size=4, iteration_num=10)
                

Experiment: 1


<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append metho

Experiment: 2


<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append metho

Experiment: 3


...

<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append metho

Experiment: 27
Experiment: 28
Experiment: 29


<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append metho

Experiment: 30


<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-7-a72a63f34701>:53: FutureWarning: The frame.append metho

### Random Draw Baseline

In [8]:
def run_experiment(data, features, target, desired_strength, experiment_num=30, initial_sample_size=0, iteration_num=10):
    # Initialize Random Forest model
    dtr = RandomForestRegressor()

    # Define scaler outside the loop
    X_scaler = StandardScaler()

    for experiment in range(experiment_num):
        print(f"Experiment: {experiment+1}")
        # Randomly sample initial training set
        train_data = data.sample(n=initial_sample_size)

        # All other data is potential test data
        test_data = data.drop(train_data.index)

        for _ in range(iteration_num):
            # Fit and transform the data
            # Randomly select a point from the test set
            original_index = np.random.choice(test_data.index)

            # Append X_next to training data
            train_data = train_data.append(test_data.loc[original_index])

            # Remove X_next from test_data
            test_data = test_data.drop(original_index)

            # If we have found a material with the desired strength, we stop sampling
            if train_data.loc[original_index, target] >= desired_strength:
                break
        filename = f"results/RP_ID/experiment_{experiment+1}_RP_experiment_{experiment+1}_target_{int(targ_quant.value)}_%_Dev_Budget_{budget}_{timestamp}.csv"
        train_data.to_csv(filename, index=False)
        
        
# load the data
file_path = os.path.join('Data', 'numeric_data.csv')
data = pd.read_csv(file_path)

# Extract features and target
features = ["Powderkg", "Liquidkg", "WC", "Fly_Ash_ratio", "GGBFS_ratio", "temperature"]
target = "fc_28dGroundTruth"
timestamp = str(int(time.time()))               
run_experiment(data, features, target, desired_strength, experiment_num=30, initial_sample_size=0, iteration_num=10)
                

<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append metho

Experiment: 1
Experiment: 2
Experiment: 3


<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append metho

Experiment: 4
Experiment: 5
Experiment: 6
Experiment: 7


<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append metho

...

<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append metho

Experiment: 24
Experiment: 25
Experiment: 26
Experiment: 27


<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append metho

Experiment: 28
Experiment: 29
Experiment: 30


<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
<ipython-input-8-c49f24cc9603>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(test_data.loc[original_index])
